In [1]:
from keras.models import Model, load_model, model_from_json
from keras.layers import Input, Cropping2D
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, History
from keras import backend as K
from keras.optimizers import Nadam
from keras.utils import multi_gpu_model

import tensorflow as tf

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
list_gpus = K.tensorflow_backend._get_available_gpus()
print(list_gpus)


['/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1']


In [3]:
import threading

In [4]:
import numpy as np
#from sklearn.model_selection import StratifiedShuffleSplit,ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [5]:
import sys
sys.path.insert(0, '/nfs/src')
from features import geopatches

In [6]:
import datetime
import os

In [7]:
import pandas as pd
import h5py

In [8]:
# Setup data seed
np.random.seed(3254)

In [9]:
# https://github.com/ternaus/kaggle_dstl_submission/

smooth = 1e-12

def jaccard_coef(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_int(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))

    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_loss(y_true, y_pred):
    return -K.log(jaccard_coef(y_true, y_pred)) + K.binary_crossentropy(y_pred, y_true)
#    return -K.log(jaccard_coef(y_true, y_pred))

def save_model(model, cross, results_dir):
    json_string = model.to_json()
    if not os.path.isdir(os.path.join(results_dir,'cache')):
        os.makedirs(os.path.join(results_dir,'cache'))
    json_name = 'architecture_' + cross + '.json'
    weight_name = 'model_weights_' + cross + '.h5'
    open(os.path.join(results_dir,'cache', json_name), 'w').write(json_string)
    model.save_weights(os.path.join(results_dir,'cache', weight_name), overwrite=True)

def save_history(history, cross, results_dir):
    if not os.path.isdir(os.path.join(results_dir,'history')):
        os.makedirs(os.path.join(results_dir,'history'))
    filename = os.path.join(results_dir,'history') + '/history_' + cross + '.csv'
    pd.DataFrame(history.history).to_csv(filename, index=False)

def read_model(cross,results_dir):
    json_name = 'architecture_' + cross + '.json'
    weight_name = 'model_weights_' + cross + '.h5'
    model = model_from_json(open(os.path.join(results_dir,'cache', json_name)).read())
    model.load_weights(os.path.join(results_dir,'cache', weight_name))
    return model

In [10]:
def model_unet(img_cols, img_rows, nb_channels, img_crop):
    
    input_shape = (img_cols, img_rows, nb_channels)
    inputs = Input(input_shape)

    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (inputs)
    c1 = BatchNormalization(axis=1)(c1)
    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (c1)
    c1 = BatchNormalization(axis=1)(c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (p1)
    c2 = BatchNormalization(axis=1)(c2)
    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (c2)
    c2 = BatchNormalization(axis=1)(c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (p2)
    c3 = BatchNormalization(axis=1)(c3)
    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (c3)
    c3 = BatchNormalization(axis=1)(c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (p3)
    c4 = BatchNormalization(axis=1)(c4)
    c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (c4)
    c4 = BatchNormalization(axis=1)(c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(128, (3, 3), activation='elu', padding='same') (p4)
    c5 = BatchNormalization(axis=1)(c5)
    c5 = Conv2D(128, (3, 3), activation='elu', padding='same') (c5)
    c5 = BatchNormalization(axis=1)(c5)

    u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (u6)
    c6 = BatchNormalization(axis=1)(c6)
    c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (c6)
    c6 = BatchNormalization(axis=1)(c6)

    u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (u7)
    c7 = BatchNormalization(axis=1)(c7)
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (c7)
    c7 = BatchNormalization(axis=1)(c7)

    u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (u8)
    c8 = BatchNormalization(axis=1)(c8)
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (c8)
    c8 = BatchNormalization(axis=1)(c8)

    u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
    c9 = BatchNormalization(axis=1)(c9)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)
    c9 = BatchNormalization(axis=1)(c9)

    #outputs = Conv2D(25, (1, 1), activation='sigmoid') (c9)
    c10 = Conv2D(1, (1, 1), activation='sigmoid') (c9)

    outputs = Cropping2D(cropping=((img_crop, img_crop), (img_crop, img_crop))) (c10)
    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

In [11]:
#https://keunwoochoi.wordpress.com/2017/08/24/tip-fit_generator-in-keras-how-to-parallelise-correctly/

class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()
    
    def next(self):
        with self.lock:
            #return self.it.next()
            return next(self.it)


def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g

@threadsafe_generator
def generate_minibatch(X_train, Y_train, batch_size):
    while 1:
        # Generate one minibatch
        X_batch = np.empty((batch_size, X_train.shape[1], X_train.shape[2], X_train.shape[3]))
        Y_batch = np.empty((batch_size, Y_train.shape[1], Y_train.shape[2], Y_train.shape[3]))
        for i in range(batch_size):
            # Select a random image
            idx = np.random.choice(X_train.shape[0])
            image = X_train[idx]
            label = Y_train[idx]

            if np.random.choice(2) == 0:
                # Flip vertically
                image = np.flipud(image)
                label = np.flipud(label)
            if np.random.choice(2) == 0:
                # Flip horizontally
                image = np.fliplr(image)
                label = np.fliplr(label)

            num_rot = np.random.choice(4)
            image = np.rot90(image, num_rot)
            label = np.rot90(label, num_rot)

            X_batch[i] = image
            Y_batch[i] = label

        yield (X_batch, Y_batch)

In [12]:
def encode_binary_data(dict_replace):
    # Encode classes
    print(dict_replace)
    u,inv = np.unique(Y_raw,return_inverse = True)
    Y = np.array([dict_replace.get(x, 0.0) for x in u])[inv].reshape(Y_raw.shape)
    Y = Y[..., np.newaxis]
    X = X_raw
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=3254)
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.10, random_state=3254)
    
    for data,name in zip([Y_train,Y_val,Y_test],['Y_train','Y_val','Y_test']):
        num_img = 0.0
        num_pixels = 0.0
        for x in range(data.shape[0]):
            unique, counts = np.unique(data[x], return_counts=True)
            if len(unique) > 1:
                #print(x, dict(zip(unique, counts)))
                num_img += 1
                num_pixels += dict(zip(unique, counts))[1]
        print ('composition with label != 0')
        print (name, num_img/data.shape[0], num_pixels/(data.shape[0] * data.shape[1] * data.shape[2]))
    return X_train, X_val, X_test, Y_train, Y_val, Y_test

In [13]:
if __name__ == '__main__':
    
    results_dir = '/nfs/results/unet/extrabandtest/80_80_16/results_prelim_unet_pl_veg'
    
    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/US_140EVT_020818_UTM11_30_clipped.tif'],dict_label,\
                                       extraindexbands=[{'type': 'evi', 'nir':4, 'red': 2, 'blue': 0},\
                                                        {'type': 'ccci', 'nir':4, 'red': 2, 'rededge': 3},\
                                                        {'type': 'savi', 'nir':4, 'red': 2, 'L': 0.5}],\
                                         normalizebands=65535,
                                          repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    #X_raw = X_raw / 65535
    # Done in create_done now with normalizebands
    
    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')

    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')
        

unique labels found:
[3014 3092 3096 3097 3099 3110 3118 3129 3130 3152 3155 3181 3182 3183
 3184 3221 3292 3294 3296 3297 3298 3299 3900 3901 3902 3903 3904 3910
 3911 3912 3913 3914 3923 3924 3926 3927 3928 3929 3964 3984]
labels that do not meet threshold
[3014, 3096, 3155, 3181, 3182, 3183, 3221, 3901, 3911, 3923, 3924, 3926, 3927, 3928, 3964, 3984]
final labels
[3092, 3097, 3099, 3110, 3118, 3129, 3130, 3152, 3184, 3292, 3294, 3296, 3297, 3298, 3299, 3900, 3902, 3903, 3904, 3910, 3912, 3913, 3914, 3929]


label: 3092
{3092: 1.0}
composition with label != 0
Y_train 0.5410628019323671 0.1474718196457327
composition with label != 0
Y_val 0.6 0.19891964285714286
composition with label != 0
Y_test 0.5202312138728323 0.12354407514450867
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 8) (70, 112, 112, 8) (173, 112, 112, 8) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:61: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:61: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., validation_data=(array([[[..., epochs=50, callbacks=[<keras.ca..., steps_per_epoch=100, verbose=1)`


Epoch 1/50
100/100 [==============================] - 26s 259ms/step - loss: 4.8058 - binary_crossentropy: 0.4865 - jaccard_coef_int: 0.2002 - binary_accuracy: 0.8362 - val_loss: 5.3543 - val_binary_crossentropy: 1.2913 - val_jaccard_coef_int: 0.1813 - val_binary_accuracy: 0.7789
Epoch 2/50
100/100 [==============================] - 18s 179ms/step - loss: 3.9743 - binary_crossentropy: 0.5539 - jaccard_coef_int: 0.2826 - binary_accuracy: 0.8442 - val_loss: 7.9534 - val_binary_crossentropy: 0.9456 - val_jaccard_coef_int: 8.9992e-16 - val_binary_accuracy: 0.8011
Epoch 3/50
100/100 [==============================] - 18s 182ms/step - loss: 3.6019 - binary_crossentropy: 0.5936 - jaccard_coef_int: 0.3089 - binary_accuracy: 0.8567 - val_loss: 4.3524 - val_binary_crossentropy: 1.3665 - val_jaccard_coef_int: 0.2885 - val_binary_accuracy: 0.8090
Epoch 4/50
100/100 [==============================] - 18s 181ms/step - loss: 3.3529 - binary_crossentropy: 0.6412 - jaccard_coef_int: 0.3461 - binary_acc

Epoch 7/50
100/100 [==============================] - 18s 184ms/step - loss: 2.4124 - binary_crossentropy: 0.0663 - jaccard_coef_int: 0.1271 - binary_accuracy: 0.9895 - val_loss: 8.1094 - val_binary_crossentropy: 0.0879 - val_jaccard_coef_int: 1.8701e-14 - val_binary_accuracy: 0.9890
Epoch 8/50
100/100 [==============================] - 18s 181ms/step - loss: 2.3774 - binary_crossentropy: 0.0700 - jaccard_coef_int: 0.1301 - binary_accuracy: 0.9894 - val_loss: 5.8377 - val_binary_crossentropy: 0.0954 - val_jaccard_coef_int: 0.0036 - val_binary_accuracy: 0.9880
Epoch 9/50
100/100 [==============================] - 18s 184ms/step - loss: 2.4735 - binary_crossentropy: 0.0790 - jaccard_coef_int: 0.1205 - binary_accuracy: 0.9883 - val_loss: 7.2191 - val_binary_crossentropy: 0.0982 - val_jaccard_coef_int: 1.6815e-14 - val_binary_accuracy: 0.9880
Epoch 10/50
100/100 [==============================] - 18s 182ms/step - loss: 2.2300 - binary_crossentropy: 0.0672 - jaccard_coef_int: 0.1404 - binar

Epoch 36/50
100/100 [==============================] - 19s 185ms/step - loss: 2.2363 - binary_crossentropy: 0.1099 - jaccard_coef_int: 0.1391 - binary_accuracy: 0.9905 - val_loss: 2.8648 - val_binary_crossentropy: 0.1771 - val_jaccard_coef_int: 0.0734 - val_binary_accuracy: 0.9842
Epoch 37/50
100/100 [==============================] - 18s 182ms/step - loss: 2.1021 - binary_crossentropy: 0.1040 - jaccard_coef_int: 0.1535 - binary_accuracy: 0.9910 - val_loss: 3.4058 - val_binary_crossentropy: 0.1550 - val_jaccard_coef_int: 0.0411 - val_binary_accuracy: 0.9868
Epoch 38/50
100/100 [==============================] - 18s 184ms/step - loss: 1.9163 - binary_crossentropy: 0.0932 - jaccard_coef_int: 0.1801 - binary_accuracy: 0.9920 - val_loss: 3.2067 - val_binary_crossentropy: 0.1960 - val_jaccard_coef_int: 0.0530 - val_binary_accuracy: 0.9836
Epoch 39/50
100/100 [==============================] - 18s 182ms/step - loss: 2.0335 - binary_crossentropy: 0.1049 - jaccard_coef_int: 0.1634 - binary_acc

...

Epoch 26/50
100/100 [==============================] - 16s 156ms/step - loss: 2.0025 - binary_crossentropy: 0.2634 - jaccard_coef_int: 0.2112 - binary_accuracy: 0.9737 - val_loss: 2.0166 - val_binary_crossentropy: 0.1526 - val_jaccard_coef_int: 0.1718 - val_binary_accuracy: 0.9850
Epoch 27/50
100/100 [==============================] - 16s 161ms/step - loss: 1.9656 - binary_crossentropy: 0.2682 - jaccard_coef_int: 0.2195 - binary_accuracy: 0.9735 - val_loss: 2.4564 - val_binary_crossentropy: 0.1201 - val_jaccard_coef_int: 0.1068 - val_binary_accuracy: 0.9859
Epoch 28/50
100/100 [==============================] - 16s 158ms/step - loss: 1.9886 - binary_crossentropy: 0.2679 - jaccard_coef_int: 0.2127 - binary_accuracy: 0.9739 - val_loss: 2.3054 - val_binary_crossentropy: 0.1149 - val_jaccard_coef_int: 0.1237 - val_binary_accuracy: 0.9873
Epoch 29/50
100/100 [==============================] - 16s 158ms/step - loss: 1.9669 - binary_crossentropy: 0.2756 - jaccard_coef_int: 0.2185 - binary_acc

Epoch 3/50
100/100 [==============================] - 16s 156ms/step - loss: 3.3283 - binary_crossentropy: 0.1638 - jaccard_coef_int: 0.0799 - binary_accuracy: 0.9596 - val_loss: 4.0880 - val_binary_crossentropy: 0.1206 - val_jaccard_coef_int: 0.0244 - val_binary_accuracy: 0.9802
Epoch 4/50
100/100 [==============================] - 15s 154ms/step - loss: 3.1713 - binary_crossentropy: 0.1966 - jaccard_coef_int: 0.0867 - binary_accuracy: 0.9604 - val_loss: 3.3020 - val_binary_crossentropy: 0.1806 - val_jaccard_coef_int: 0.0630 - val_binary_accuracy: 0.9705
Epoch 5/50
100/100 [==============================] - 15s 154ms/step - loss: 3.0378 - binary_crossentropy: 0.2033 - jaccard_coef_int: 0.0912 - binary_accuracy: 0.9643 - val_loss: 3.3180 - val_binary_crossentropy: 0.1520 - val_jaccard_coef_int: 0.0568 - val_binary_accuracy: 0.9741
Epoch 6/50
100/100 [==============================] - 16s 156ms/step - loss: 3.0644 - binary_crossentropy: 0.2209 - jaccard_coef_int: 0.0895 - binary_accurac

Epoch 9/50
100/100 [==============================] - 12s 119ms/step - loss: 3.7154 - binary_crossentropy: 0.5697 - jaccard_coef_int: 0.1002 - binary_accuracy: 0.9149 - val_loss: 4.0267 - val_binary_crossentropy: 0.5330 - val_jaccard_coef_int: 0.0575 - val_binary_accuracy: 0.9275
Epoch 10/50
100/100 [==============================] - 12s 116ms/step - loss: 3.5031 - binary_crossentropy: 0.6021 - jaccard_coef_int: 0.1147 - binary_accuracy: 0.9187 - val_loss: 4.1337 - val_binary_crossentropy: 0.5707 - val_jaccard_coef_int: 0.0556 - val_binary_accuracy: 0.9223
Epoch 11/50
100/100 [==============================] - 12s 118ms/step - loss: 3.5602 - binary_crossentropy: 0.6410 - jaccard_coef_int: 0.1126 - binary_accuracy: 0.9163 - val_loss: 4.0862 - val_binary_crossentropy: 0.6404 - val_jaccard_coef_int: 0.0636 - val_binary_accuracy: 0.9173
Epoch 12/50
100/100 [==============================] - 12s 117ms/step - loss: 3.4323 - binary_crossentropy: 0.6257 - jaccard_coef_int: 0.1222 - binary_accu

Epoch 15/50
100/100 [==============================] - 16s 156ms/step - loss: 3.5097 - binary_crossentropy: 0.6465 - jaccard_coef_int: 0.1038 - binary_accuracy: 0.9236 - val_loss: 4.4774 - val_binary_crossentropy: 0.6349 - val_jaccard_coef_int: 0.0368 - val_binary_accuracy: 0.9266
Epoch 16/50
100/100 [==============================] - 16s 157ms/step - loss: 3.5193 - binary_crossentropy: 0.6615 - jaccard_coef_int: 0.1039 - binary_accuracy: 0.9227 - val_loss: 4.0439 - val_binary_crossentropy: 0.6599 - val_jaccard_coef_int: 0.0584 - val_binary_accuracy: 0.9248
Epoch 17/50
100/100 [==============================] - 15s 155ms/step - loss: 3.4806 - binary_crossentropy: 0.6518 - jaccard_coef_int: 0.1032 - binary_accuracy: 0.9257 - val_loss: 4.5251 - val_binary_crossentropy: 0.6219 - val_jaccard_coef_int: 0.0340 - val_binary_accuracy: 0.9286
Epoch 18/50
100/100 [==============================] - 16s 156ms/step - loss: 3.4639 - binary_crossentropy: 0.6722 - jaccard_coef_int: 0.1052 - binary_acc

Epoch 21/50
100/100 [==============================] - 16s 158ms/step - loss: 2.3418 - binary_crossentropy: 0.0972 - jaccard_coef_int: 0.1636 - binary_accuracy: 0.9881 - val_loss: 5.1695 - val_binary_crossentropy: 0.0204 - val_jaccard_coef_int: 0.4787 - val_binary_accuracy: 0.9976
Epoch 22/50
100/100 [==============================] - 16s 157ms/step - loss: 2.2158 - binary_crossentropy: 0.1037 - jaccard_coef_int: 0.1916 - binary_accuracy: 0.9876 - val_loss: 5.4039 - val_binary_crossentropy: 0.0174 - val_jaccard_coef_int: 0.4672 - val_binary_accuracy: 0.9978
Epoch 23/50
100/100 [==============================] - 16s 162ms/step - loss: 2.5595 - binary_crossentropy: 0.1068 - jaccard_coef_int: 0.1680 - binary_accuracy: 0.9863 - val_loss: 5.6134 - val_binary_crossentropy: 0.1050 - val_jaccard_coef_int: 0.0056 - val_binary_accuracy: 0.9846
Epoch 24/50
100/100 [==============================] - 16s 161ms/step - loss: 2.8466 - binary_crossentropy: 0.1360 - jaccard_coef_int: 0.1266 - binary_acc

In [14]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/extrabandtest/80_80_16/results_prelim_unet_pl_fire'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/US_140FBFM13_3030_clipped.tif'],dict_label,\
                                       extraindexbands=[{'type': 'evi', 'nir':4, 'red': 2, 'blue': 0},\
                                                        {'type': 'ccci', 'nir':4, 'red': 2, 'rededge': 3},\
                                                        {'type': 'savi', 'nir':4, 'red': 2, 'L': 0.5}],\
                                         normalizebands=65535,
                                          repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    #X_raw = X_raw / 65535
    # Done in create_done now with normalizebands

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[ 1  2  4  5  7  8  9 91 93 98 99]
labels that do not meet threshold
[7, 93]
final labels
[1, 2, 4, 5, 8, 9, 91, 98, 99]


label: 1
{1: 1.0}
composition with label != 0
Y_train 0.8582930756843801 0.2713395732689211
composition with label != 0
Y_val 0.8428571428571429 0.2126607142857143
composition with label != 0
Y_test 0.861271676300578 0.2920628612716763
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 8) (70, 112, 112, 8) (173, 112, 112, 8) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:60: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:60: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., validation_data=(array([[[..., epochs=50, callbacks=[<keras.ca..., steps_per_epoch=100, verbose=1)`


Epoch 1/50
100/100 [==============================] - 51s 508ms/step - loss: 5.3397 - binary_crossentropy: 0.6952 - jaccard_coef_int: 0.3233 - binary_accuracy: 0.7641 - val_loss: 4.4352 - val_binary_crossentropy: 1.0139 - val_jaccard_coef_int: 0.2696 - val_binary_accuracy: 0.8075
Epoch 2/50
100/100 [==============================] - 12s 118ms/step - loss: 4.5259 - binary_crossentropy: 0.8940 - jaccard_coef_int: 0.3854 - binary_accuracy: 0.7848 - val_loss: 4.4736 - val_binary_crossentropy: 0.9403 - val_jaccard_coef_int: 0.3076 - val_binary_accuracy: 0.7986
Epoch 3/50
100/100 [==============================] - 12s 117ms/step - loss: 4.3328 - binary_crossentropy: 0.9895 - jaccard_coef_int: 0.3998 - binary_accuracy: 0.7912 - val_loss: 4.1019 - val_binary_crossentropy: 1.1210 - val_jaccard_coef_int: 0.3771 - val_binary_accuracy: 0.8066
Epoch 4/50
100/100 [==============================] - 12s 117ms/step - loss: 4.2852 - binary_crossentropy: 1.0751 - jaccard_coef_int: 0.3997 - binary_accurac

Epoch 7/50
100/100 [==============================] - 12s 120ms/step - loss: 3.9224 - binary_crossentropy: 0.7425 - jaccard_coef_int: 0.1552 - binary_accuracy: 0.8757 - val_loss: 4.4004 - val_binary_crossentropy: 0.7182 - val_jaccard_coef_int: 0.0857 - val_binary_accuracy: 0.8798
Epoch 8/50
100/100 [==============================] - 12s 121ms/step - loss: 3.7802 - binary_crossentropy: 0.7416 - jaccard_coef_int: 0.1623 - binary_accuracy: 0.8808 - val_loss: 4.9655 - val_binary_crossentropy: 1.1256 - val_jaccard_coef_int: 0.1308 - val_binary_accuracy: 0.8183
Epoch 9/50
100/100 [==============================] - 12s 120ms/step - loss: 3.6905 - binary_crossentropy: 0.7619 - jaccard_coef_int: 0.1648 - binary_accuracy: 0.8850 - val_loss: 4.9574 - val_binary_crossentropy: 0.9213 - val_jaccard_coef_int: 0.0602 - val_binary_accuracy: 0.8656
Epoch 10/50
100/100 [==============================] - 12s 120ms/step - loss: 3.6864 - binary_crossentropy: 0.7844 - jaccard_coef_int: 0.1642 - binary_accura

Epoch 13/50
100/100 [==============================] - 16s 157ms/step - loss: 2.3095 - binary_crossentropy: 0.1443 - jaccard_coef_int: 0.1348 - binary_accuracy: 0.9842 - val_loss: 3.2241 - val_binary_crossentropy: 0.2211 - val_jaccard_coef_int: 0.0576 - val_binary_accuracy: 0.9771
Epoch 14/50
100/100 [==============================] - 16s 158ms/step - loss: 2.2173 - binary_crossentropy: 0.1510 - jaccard_coef_int: 0.1459 - binary_accuracy: 0.9841 - val_loss: 3.0609 - val_binary_crossentropy: 0.2589 - val_jaccard_coef_int: 0.0711 - val_binary_accuracy: 0.9740
Epoch 15/50
100/100 [==============================] - 16s 160ms/step - loss: 2.3237 - binary_crossentropy: 0.1537 - jaccard_coef_int: 0.1340 - binary_accuracy: 0.9840 - val_loss: 2.9992 - val_binary_crossentropy: 0.3429 - val_jaccard_coef_int: 0.0846 - val_binary_accuracy: 0.9672
Epoch 16/50
100/100 [==============================] - 16s 160ms/step - loss: 2.1530 - binary_crossentropy: 0.1529 - jaccard_coef_int: 0.1559 - binary_acc

...

Epoch 2/50
100/100 [==============================] - 17s 167ms/step - loss: 2.5885 - binary_crossentropy: 0.1399 - jaccard_coef_int: 0.1880 - binary_accuracy: 0.9641 - val_loss: 3.6070 - val_binary_crossentropy: 0.0864 - val_jaccard_coef_int: 0.0425 - val_binary_accuracy: 0.9818
Epoch 3/50
100/100 [==============================] - 16s 160ms/step - loss: 2.3699 - binary_crossentropy: 0.1555 - jaccard_coef_int: 0.1984 - binary_accuracy: 0.9645 - val_loss: 2.9499 - val_binary_crossentropy: 0.1512 - val_jaccard_coef_int: 0.0906 - val_binary_accuracy: 0.9719
Epoch 4/50
100/100 [==============================] - 17s 167ms/step - loss: 2.3234 - binary_crossentropy: 0.1745 - jaccard_coef_int: 0.1958 - binary_accuracy: 0.9644 - val_loss: 2.8846 - val_binary_crossentropy: 0.1864 - val_jaccard_coef_int: 0.0932 - val_binary_accuracy: 0.9714
Epoch 5/50
100/100 [==============================] - 16s 162ms/step - loss: 2.1614 - binary_crossentropy: 0.2035 - jaccard_coef_int: 0.2186 - binary_accurac

Epoch 8/50
100/100 [==============================] - 18s 182ms/step - loss: 2.9474 - binary_crossentropy: 0.8686 - jaccard_coef_int: 0.6075 - binary_accuracy: 0.8486 - val_loss: 3.0454 - val_binary_crossentropy: 0.9312 - val_jaccard_coef_int: 0.6211 - val_binary_accuracy: 0.8412
Epoch 9/50
100/100 [==============================] - 17s 167ms/step - loss: 2.9230 - binary_crossentropy: 0.8955 - jaccard_coef_int: 0.6088 - binary_accuracy: 0.8498 - val_loss: 2.9135 - val_binary_crossentropy: 0.9277 - val_jaccard_coef_int: 0.6089 - val_binary_accuracy: 0.8500
Epoch 10/50
100/100 [==============================] - 17s 169ms/step - loss: 3.0652 - binary_crossentropy: 0.9232 - jaccard_coef_int: 0.5980 - binary_accuracy: 0.8422 - val_loss: 2.9049 - val_binary_crossentropy: 0.9117 - val_jaccard_coef_int: 0.6049 - val_binary_accuracy: 0.8506
Epoch 11/50
100/100 [==============================] - 17s 175ms/step - loss: 2.9263 - binary_crossentropy: 0.9259 - jaccard_coef_int: 0.6173 - binary_accur

Epoch 14/50
100/100 [==============================] - 17s 175ms/step - loss: 0.6532 - binary_crossentropy: 0.0474 - jaccard_coef_int: 0.6374 - binary_accuracy: 0.9931 - val_loss: 1.1051 - val_binary_crossentropy: 0.1092 - val_jaccard_coef_int: 0.4227 - val_binary_accuracy: 0.9856
Epoch 15/50
100/100 [==============================] - 17s 174ms/step - loss: 0.8371 - binary_crossentropy: 0.0610 - jaccard_coef_int: 0.6251 - binary_accuracy: 0.9897 - val_loss: 1.0656 - val_binary_crossentropy: 0.1134 - val_jaccard_coef_int: 0.4306 - val_binary_accuracy: 0.9863
Epoch 16/50
100/100 [==============================] - 18s 178ms/step - loss: 0.6035 - binary_crossentropy: 0.0422 - jaccard_coef_int: 0.6675 - binary_accuracy: 0.9936 - val_loss: 1.1029 - val_binary_crossentropy: 0.1074 - val_jaccard_coef_int: 0.4191 - val_binary_accuracy: 0.9861
Epoch 17/50
100/100 [==============================] - 18s 182ms/step - loss: 0.6469 - binary_crossentropy: 0.0516 - jaccard_coef_int: 0.6422 - binary_acc

Epoch 20/50
100/100 [==============================] - 17s 166ms/step - loss: 4.0917 - binary_crossentropy: 0.1508 - jaccard_coef_int: 0.0427 - binary_accuracy: 0.9682 - val_loss: 2.8918 - val_binary_crossentropy: 0.1671 - val_jaccard_coef_int: 0.0964 - val_binary_accuracy: 0.9666
Epoch 21/50
100/100 [==============================] - 17s 168ms/step - loss: 4.2110 - binary_crossentropy: 0.1613 - jaccard_coef_int: 0.0375 - binary_accuracy: 0.9672 - val_loss: 3.4385 - val_binary_crossentropy: 0.2052 - val_jaccard_coef_int: 0.0641 - val_binary_accuracy: 0.9590
Epoch 22/50
100/100 [==============================] - 17s 168ms/step - loss: 4.0583 - binary_crossentropy: 0.1354 - jaccard_coef_int: 0.0403 - binary_accuracy: 0.9739 - val_loss: 3.1107 - val_binary_crossentropy: 0.1738 - val_jaccard_coef_int: 0.0780 - val_binary_accuracy: 0.9675
Epoch 23/50
100/100 [==============================] - 17s 167ms/step - loss: 4.0169 - binary_crossentropy: 0.1543 - jaccard_coef_int: 0.0442 - binary_acc

100/100 [==============================] - 17s 166ms/step - loss: 3.8799 - binary_crossentropy: 0.2383 - jaccard_coef_int: 0.0545 - binary_accuracy: 0.9712 - val_loss: 3.1683 - val_binary_crossentropy: 0.3637 - val_jaccard_coef_int: 0.0737 - val_binary_accuracy: 0.9655
Epoch 50/50
100/100 [==============================] - 16s 165ms/step - loss: 4.1259 - binary_crossentropy: 0.1760 - jaccard_coef_int: 0.0340 - binary_accuracy: 0.9734 - val_loss: 3.0058 - val_binary_crossentropy: 0.1475 - val_jaccard_coef_int: 0.0835 - val_binary_accuracy: 0.9706
0.9845366690751445
             precision    recall  f1-score   support

        0.0       0.99      0.99      0.99   1100808
        1.0       0.00      0.01      0.01      6392

avg / total       0.99      0.98      0.99   1107200





In [13]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/extrabandtest/80_80_16/results_prelim_unet_pl_landcover'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/escondido_landcover_clipped.tif'],dict_label,\
                                       extraindexbands=[{'type': 'evi', 'nir':4, 'red': 2, 'blue': 0},\
                                                        {'type': 'ccci', 'nir':4, 'red': 2, 'rededge': 3},\
                                                        {'type': 'savi', 'nir':4, 'red': 2, 'L': 0.5}],\
                                         normalizebands=65535,
                                          repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    #X_raw = X_raw / 65535
    # Done in create_done now with normalizebands

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[11 21 22 23 24 31 42 43 52 71 81 82 90 95]
labels that do not meet threshold
[42, 81, 95]
final labels
[11, 21, 22, 23, 24, 31, 43, 52, 71, 82, 90]


label: 11
{11: 1.0}
composition with label != 0
Y_train 0.06924315619967794 0.017379227053140097
composition with label != 0
Y_val 0.05714285714285714 0.019285714285714285
composition with label != 0
Y_test 0.046242774566473986 0.011065751445086706
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 8) (70, 112, 112, 8) (173, 112, 112, 8) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:60: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:60: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., callbacks=[<keras.ca..., epochs=50, verbose=1, steps_per_epoch=100, validation_data=(array([[[...)`


Epoch 1/50
100/100 [==============================] - 25s 251ms/step - loss: 2.5994 - binary_crossentropy: 0.0763 - jaccard_coef_int: 0.6269 - binary_accuracy: 0.9861 - val_loss: 1.2116 - val_binary_crossentropy: 0.0831 - val_jaccard_coef_int: 0.6408 - val_binary_accuracy: 0.9901
Epoch 2/50
100/100 [==============================] - 18s 177ms/step - loss: 0.9070 - binary_crossentropy: 0.0260 - jaccard_coef_int: 0.7280 - binary_accuracy: 0.9951 - val_loss: 0.7818 - val_binary_crossentropy: 0.0643 - val_jaccard_coef_int: 0.6486 - val_binary_accuracy: 0.9906
Epoch 3/50
100/100 [==============================] - 18s 180ms/step - loss: 1.6630 - binary_crossentropy: 0.0340 - jaccard_coef_int: 0.6168 - binary_accuracy: 0.9933 - val_loss: 0.7705 - val_binary_crossentropy: 0.0690 - val_jaccard_coef_int: 0.6199 - val_binary_accuracy: 0.9900
Epoch 4/50
100/100 [==============================] - 19s 189ms/step - loss: 0.4996 - binary_crossentropy: 0.0246 - jaccard_coef_int: 0.7556 - binary_accurac

100/100 [==============================] - 18s 185ms/step - loss: 0.3370 - binary_crossentropy: 0.0349 - jaccard_coef_int: 0.7724 - binary_accuracy: 0.9961 - val_loss: 0.5488 - val_binary_crossentropy: 0.0952 - val_jaccard_coef_int: 0.6684 - val_binary_accuracy: 0.9911
Epoch 31/50
100/100 [==============================] - 19s 185ms/step - loss: 0.3015 - binary_crossentropy: 0.0373 - jaccard_coef_int: 0.7965 - binary_accuracy: 0.9961 - val_loss: 0.5400 - val_binary_crossentropy: 0.1203 - val_jaccard_coef_int: 0.6708 - val_binary_accuracy: 0.9913
Epoch 32/50
100/100 [==============================] - 18s 182ms/step - loss: 0.2864 - binary_crossentropy: 0.0356 - jaccard_coef_int: 0.8038 - binary_accuracy: 0.9962 - val_loss: 0.5354 - val_binary_crossentropy: 0.1000 - val_jaccard_coef_int: 0.6741 - val_binary_accuracy: 0.9914
Epoch 33/50
100/100 [==============================] - 18s 182ms/step - loss: 0.2931 - binary_crossentropy: 0.0368 - jaccard_coef_int: 0.7999 - binary_accuracy: 0.996

Epoch 7/50
100/100 [==============================] - 19s 186ms/step - loss: 4.5588 - binary_crossentropy: 1.1941 - jaccard_coef_int: 0.3002 - binary_accuracy: 0.7920 - val_loss: 4.5463 - val_binary_crossentropy: 1.2174 - val_jaccard_coef_int: 0.2700 - val_binary_accuracy: 0.7988
Epoch 8/50
100/100 [==============================] - 19s 187ms/step - loss: 4.5718 - binary_crossentropy: 1.2609 - jaccard_coef_int: 0.3060 - binary_accuracy: 0.7896 - val_loss: 4.5990 - val_binary_crossentropy: 1.2985 - val_jaccard_coef_int: 0.3001 - val_binary_accuracy: 0.7887
Epoch 9/50
100/100 [==============================] - 18s 185ms/step - loss: 4.3451 - binary_crossentropy: 1.4488 - jaccard_coef_int: 0.3230 - binary_accuracy: 0.7998 - val_loss: 4.5263 - val_binary_crossentropy: 1.4664 - val_jaccard_coef_int: 0.2789 - val_binary_accuracy: 0.7975
Epoch 15/50
100/100 [==============================] - 19s 186ms/step - loss: 4.2833 - binary_crossentropy: 1.4562 - jaccard_coef_int: 0.3253 - binary_accura

...

Epoch 16/50
100/100 [==============================] - 15s 152ms/step - loss: 2.3442 - binary_crossentropy: 0.5755 - jaccard_coef_int: 0.2832 - binary_accuracy: 0.9333 - val_loss: 3.5282 - val_binary_crossentropy: 0.9006 - val_jaccard_coef_int: 0.1575 - val_binary_accuracy: 0.8954
Epoch 17/50
100/100 [==============================] - 15s 153ms/step - loss: 2.2845 - binary_crossentropy: 0.5549 - jaccard_coef_int: 0.2821 - binary_accuracy: 0.9371 - val_loss: 4.0123 - val_binary_crossentropy: 0.7678 - val_jaccard_coef_int: 0.0696 - val_binary_accuracy: 0.9155
Epoch 18/50
100/100 [==============================] - 15s 151ms/step - loss: 2.1848 - binary_crossentropy: 0.5535 - jaccard_coef_int: 0.3062 - binary_accuracy: 0.9382 - val_loss: 3.8180 - val_binary_crossentropy: 0.8858 - val_jaccard_coef_int: 0.1077 - val_binary_accuracy: 0.9008
Epoch 19/50
100/100 [==============================] - 15s 154ms/step - loss: 2.1717 - binary_crossentropy: 0.5679 - jaccard_coef_int: 0.3119 - binary_acc

100/100 [==============================] - 15s 152ms/step - loss: 1.4857 - binary_crossentropy: 0.5207 - jaccard_coef_int: 0.4708 - binary_accuracy: 0.9545 - val_loss: 3.9786 - val_binary_crossentropy: 0.9463 - val_jaccard_coef_int: 0.0723 - val_binary_accuracy: 0.9153
Epoch 46/50
100/100 [==============================] - 15s 154ms/step - loss: 1.4740 - binary_crossentropy: 0.5069 - jaccard_coef_int: 0.4656 - binary_accuracy: 0.9559 - val_loss: 3.7009 - val_binary_crossentropy: 1.0677 - val_jaccard_coef_int: 0.1085 - val_binary_accuracy: 0.9074
Epoch 47/50
100/100 [==============================] - 15s 152ms/step - loss: 1.4411 - binary_crossentropy: 0.5040 - jaccard_coef_int: 0.4768 - binary_accuracy: 0.9565 - val_loss: 4.0681 - val_binary_crossentropy: 1.0052 - val_jaccard_coef_int: 0.0687 - val_binary_accuracy: 0.9129
Epoch 48/50
100/100 [==============================] - 15s 154ms/step - loss: 1.4428 - binary_crossentropy: 0.5098 - jaccard_coef_int: 0.4783 - binary_accuracy: 0.956

Epoch 22/50
100/100 [==============================] - 15s 152ms/step - loss: 3.4861 - binary_crossentropy: 0.1219 - jaccard_coef_int: 0.0817 - binary_accuracy: 0.9858 - val_loss: 28.0768 - val_binary_crossentropy: 6.1947e-04 - val_jaccard_coef_int: 0.9125 - val_binary_accuracy: 0.9999
Epoch 23/50
100/100 [==============================] - 15s 152ms/step - loss: 3.3774 - binary_crossentropy: 0.1069 - jaccard_coef_int: 0.0843 - binary_accuracy: 0.9855 - val_loss: 28.5144 - val_binary_crossentropy: 0.0025 - val_jaccard_coef_int: 0.7875 - val_binary_accuracy: 0.9996
Epoch 24/50
100/100 [==============================] - 15s 155ms/step - loss: 2.9078 - binary_crossentropy: 0.1075 - jaccard_coef_int: 0.0992 - binary_accuracy: 0.9878 - val_loss: 27.7971 - val_binary_crossentropy: 2.5172e-04 - val_jaccard_coef_int: 1.0000 - val_binary_accuracy: 1.0000
Epoch 25/50
100/100 [==============================] - 16s 156ms/step - loss: 3.1304 - binary_crossentropy: 0.1176 - jaccard_coef_int: 0.0982 -

0.8717837789017341
             precision    recall  f1-score   support

        0.0       0.99      0.88      0.93   1098408
        1.0       0.02      0.31      0.04      8792

avg / total       0.99      0.87      0.92   1107200



label: 90
{90: 1.0}
composition with label != 0
Y_train 0.09017713365539452 0.0038023349436392914
composition with label != 0
Y_val 0.05714285714285714 0.0023035714285714287
composition with label != 0
Y_test 0.11560693641618497 0.006325867052023122
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 8) (70, 112, 112, 8) (173, 112, 112, 8) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)
Epoch 1/50
100/100 [==============================] - 35s 349ms/step - loss: 4.9841 - binary_crossentropy: 0.0511 - jaccard_coef_int: 0.0419 - binary_accuracy: 0.9865 - val_loss: 6.2801 - val_binary_crossentropy: 0.0172 - val_jaccard_coef_int: 0.1000 - val_binary_accuracy: 0.9977
Epoch 2/50
100/100 [===========================

Epoch 28/50
100/100 [==============================] - 18s 184ms/step - loss: 1.8267 - binary_crossentropy: 0.0507 - jaccard_coef_int: 0.2117 - binary_accuracy: 0.9948 - val_loss: 10.5589 - val_binary_crossentropy: 0.0256 - val_jaccard_coef_int: 0.1000 - val_binary_accuracy: 0.9976
Epoch 29/50
100/100 [==============================] - 18s 183ms/step - loss: 1.6171 - binary_crossentropy: 0.0486 - jaccard_coef_int: 0.2364 - binary_accuracy: 0.9953 - val_loss: 10.8973 - val_binary_crossentropy: 0.0283 - val_jaccard_coef_int: 0.1000 - val_binary_accuracy: 0.9973
Epoch 30/50
100/100 [==============================] - 18s 185ms/step - loss: 1.6839 - binary_crossentropy: 0.0483 - jaccard_coef_int: 0.2270 - binary_accuracy: 0.9953 - val_loss: 11.0564 - val_binary_crossentropy: 0.0267 - val_jaccard_coef_int: 0.1000 - val_binary_accuracy: 0.9975
Epoch 31/50
100/100 [==============================] - 18s 183ms/step - loss: 1.6570 - binary_crossentropy: 0.0501 - jaccard_coef_int: 0.2324 - binary_

In [14]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/extrabandtest/80_80_16/results_prelim_unet_pl_c24pl'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/planetlabs/ClusterRaster24_PlanetLabs.tif'],dict_label,\
                                       extraindexbands=[{'type': 'evi', 'nir':4, 'red': 2, 'blue': 0},\
                                                        {'type': 'ccci', 'nir':4, 'red': 2, 'rededge': 3},\
                                                        {'type': 'savi', 'nir':4, 'red': 2, 'L': 0.5}],\
                                         normalizebands=65535)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    #X_raw = X_raw / 65535
    # Done in create_done now with normalizebands

    #remove classes that are too small
    list_remove = []
#     for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
#         if x[1] < 400*36:
#             list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
labels that do not meet threshold
[]
final labels
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


label: 0
{0: 1.0}
composition with label != 0
Y_train 0.9307568438003221 0.0945511272141707
composition with label != 0
Y_val 0.9571428571428572 0.08802008928571428
composition with label != 0
Y_test 0.9248554913294798 0.08844924132947977
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 8) (70, 112, 112, 8) (173, 112, 112, 8) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:59: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:59: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., callbacks=[<keras.ca..., epochs=50, verbose=1, steps_per_epoch=100, validation_data=(array([[[...)`


Epoch 1/50
100/100 [==============================] - 33s 333ms/step - loss: 3.3479 - binary_crossentropy: 0.3189 - jaccard_coef_int: 0.3301 - binary_accuracy: 0.8934 - val_loss: 6.6538 - val_binary_crossentropy: 0.4719 - val_jaccard_coef_int: 3.6736e-04 - val_binary_accuracy: 0.8934
Epoch 2/50
100/100 [==============================] - 16s 157ms/step - loss: 2.0457 - binary_crossentropy: 0.2535 - jaccard_coef_int: 0.4695 - binary_accuracy: 0.9301 - val_loss: 5.1041 - val_binary_crossentropy: 0.5058 - val_jaccard_coef_int: 0.0229 - val_binary_accuracy: 0.9055
Epoch 3/50
100/100 [==============================] - 15s 154ms/step - loss: 1.6825 - binary_crossentropy: 0.2425 - jaccard_coef_int: 0.5262 - binary_accuracy: 0.9407 - val_loss: 6.3972 - val_binary_crossentropy: 0.5463 - val_jaccard_coef_int: 0.0043 - val_binary_accuracy: 0.9112
Epoch 4/50
100/100 [==============================] - 16s 156ms/step - loss: 1.4997 - binary_crossentropy: 0.2321 - jaccard_coef_int: 0.5560 - binary_acc

100/100 [==============================] - 15s 155ms/step - loss: 0.6592 - binary_crossentropy: 0.1642 - jaccard_coef_int: 0.7718 - binary_accuracy: 0.9757 - val_loss: 0.5992 - val_binary_crossentropy: 0.1373 - val_jaccard_coef_int: 0.7766 - val_binary_accuracy: 0.9788
Epoch 31/50
100/100 [==============================] - 16s 156ms/step - loss: 0.6300 - binary_crossentropy: 0.1565 - jaccard_coef_int: 0.7791 - binary_accuracy: 0.9769 - val_loss: 0.5890 - val_binary_crossentropy: 0.1342 - val_jaccard_coef_int: 0.7794 - val_binary_accuracy: 0.9792
Epoch 32/50
100/100 [==============================] - 15s 154ms/step - loss: 0.6015 - binary_crossentropy: 0.1486 - jaccard_coef_int: 0.7877 - binary_accuracy: 0.9779 - val_loss: 0.5703 - val_binary_crossentropy: 0.1309 - val_jaccard_coef_int: 0.7863 - val_binary_accuracy: 0.9798
Epoch 33/50
100/100 [==============================] - 15s 154ms/step - loss: 0.5950 - binary_crossentropy: 0.1492 - jaccard_coef_int: 0.7873 - binary_accuracy: 0.978

Epoch 7/50
100/100 [==============================] - 16s 157ms/step - loss: 1.9722 - binary_crossentropy: 0.2408 - jaccard_coef_int: 0.3385 - binary_accuracy: 0.9483 - val_loss: 2.1160 - val_binary_crossentropy: 0.3066 - val_jaccard_coef_int: 0.3601 - val_binary_accuracy: 0.9360
Epoch 8/50
100/100 [==============================] - 15s 155ms/step - loss: 1.8789 - binary_crossentropy: 0.2429 - jaccard_coef_int: 0.3591 - binary_accuracy: 0.9499 - val_loss: 2.0570 - val_binary_crossentropy: 0.3001 - val_jaccard_coef_int: 0.3594 - val_binary_accuracy: 0.9395
Epoch 9/50
100/100 [==============================] - 16s 158ms/step - loss: 1.7823 - binary_crossentropy: 0.2444 - jaccard_coef_int: 0.3806 - binary_accuracy: 0.9518 - val_loss: 1.8693 - val_binary_crossentropy: 0.2593 - val_jaccard_coef_int: 0.3731 - val_binary_accuracy: 0.9474
Epoch 10/50
100/100 [==============================] - 15s 154ms/step - loss: 1.7120 - binary_crossentropy: 0.2485 - jaccard_coef_int: 0.3959 - binary_accura

...

100/100 [==============================] - 12s 122ms/step - loss: 0.5810 - binary_crossentropy: 0.1210 - jaccard_coef_int: 0.7822 - binary_accuracy: 0.9797 - val_loss: 0.4948 - val_binary_crossentropy: 0.0903 - val_jaccard_coef_int: 0.8075 - val_binary_accuracy: 0.9833
Epoch 40/50
100/100 [==============================] - 13s 129ms/step - loss: 0.5470 - binary_crossentropy: 0.1128 - jaccard_coef_int: 0.7941 - binary_accuracy: 0.9809 - val_loss: 0.4941 - val_binary_crossentropy: 0.0940 - val_jaccard_coef_int: 0.8038 - val_binary_accuracy: 0.9836
Epoch 41/50
100/100 [==============================] - 13s 135ms/step - loss: 0.5622 - binary_crossentropy: 0.1189 - jaccard_coef_int: 0.7861 - binary_accuracy: 0.9805 - val_loss: 0.5503 - val_binary_crossentropy: 0.1131 - val_jaccard_coef_int: 0.7802 - val_binary_accuracy: 0.9816
Epoch 42/50
100/100 [==============================] - 13s 133ms/step - loss: 0.5515 - binary_crossentropy: 0.1184 - jaccard_coef_int: 0.7920 - binary_accuracy: 0.980

Epoch 16/50
100/100 [==============================] - 12s 123ms/step - loss: 0.7149 - binary_crossentropy: 0.0980 - jaccard_coef_int: 0.6628 - binary_accuracy: 0.9823 - val_loss: 0.7537 - val_binary_crossentropy: 0.0779 - val_jaccard_coef_int: 0.6150 - val_binary_accuracy: 0.9854
Epoch 17/50
100/100 [==============================] - 12s 123ms/step - loss: 0.6856 - binary_crossentropy: 0.0946 - jaccard_coef_int: 0.6752 - binary_accuracy: 0.9829 - val_loss: 0.8123 - val_binary_crossentropy: 0.0825 - val_jaccard_coef_int: 0.5718 - val_binary_accuracy: 0.9852
Epoch 18/50
100/100 [==============================] - 12s 124ms/step - loss: 0.6746 - binary_crossentropy: 0.0943 - jaccard_coef_int: 0.6771 - binary_accuracy: 0.9833 - val_loss: 0.6895 - val_binary_crossentropy: 0.0707 - val_jaccard_coef_int: 0.6309 - val_binary_accuracy: 0.9870
Epoch 19/50
100/100 [==============================] - 12s 124ms/step - loss: 0.6634 - binary_crossentropy: 0.0964 - jaccard_coef_int: 0.6854 - binary_acc

100/100 [==============================] - 13s 125ms/step - loss: 0.3816 - binary_crossentropy: 0.0637 - jaccard_coef_int: 0.8038 - binary_accuracy: 0.9902 - val_loss: 0.3912 - val_binary_crossentropy: 0.0475 - val_jaccard_coef_int: 0.7684 - val_binary_accuracy: 0.9926
Epoch 46/50
100/100 [==============================] - 12s 122ms/step - loss: 0.3715 - binary_crossentropy: 0.0626 - jaccard_coef_int: 0.8076 - binary_accuracy: 0.9906 - val_loss: 0.4132 - val_binary_crossentropy: 0.0565 - val_jaccard_coef_int: 0.7598 - val_binary_accuracy: 0.9916
Epoch 47/50
100/100 [==============================] - 12s 123ms/step - loss: 0.3729 - binary_crossentropy: 0.0646 - jaccard_coef_int: 0.8081 - binary_accuracy: 0.9904 - val_loss: 0.3761 - val_binary_crossentropy: 0.0462 - val_jaccard_coef_int: 0.7732 - val_binary_accuracy: 0.9929
Epoch 48/50
100/100 [==============================] - 12s 123ms/step - loss: 0.3648 - binary_crossentropy: 0.0638 - jaccard_coef_int: 0.8115 - binary_accuracy: 0.990

Epoch 22/50
100/100 [==============================] - 12s 117ms/step - loss: 0.9988 - binary_crossentropy: 0.2494 - jaccard_coef_int: 0.7041 - binary_accuracy: 0.9606 - val_loss: 0.9462 - val_binary_crossentropy: 0.2198 - val_jaccard_coef_int: 0.7317 - val_binary_accuracy: 0.9616
Epoch 23/50
100/100 [==============================] - 12s 118ms/step - loss: 0.9816 - binary_crossentropy: 0.2458 - jaccard_coef_int: 0.7125 - binary_accuracy: 0.9609 - val_loss: 1.0033 - val_binary_crossentropy: 0.2692 - val_jaccard_coef_int: 0.7232 - val_binary_accuracy: 0.9584
Epoch 24/50
100/100 [==============================] - 12s 118ms/step - loss: 0.9536 - binary_crossentropy: 0.2388 - jaccard_coef_int: 0.7170 - binary_accuracy: 0.9622 - val_loss: 0.9333 - val_binary_crossentropy: 0.2400 - val_jaccard_coef_int: 0.7406 - val_binary_accuracy: 0.9614
Epoch 25/50
100/100 [==============================] - 12s 116ms/step - loss: 0.9464 - binary_crossentropy: 0.2400 - jaccard_coef_int: 0.7208 - binary_acc

             precision    recall  f1-score   support

        0.0       0.99      0.98      0.99    980995
        1.0       0.88      0.93      0.91    126205

avg / total       0.98      0.98      0.98   1107200



